In [1]:
#Fundamental librarys to math and stats process
import numpy as np
import numpy.random as nr
import scipy.stats as ss
import math
#data prepared
import pandas as pd

#ML preprocessi
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import sklearn.model_selection as ms
from sklearn.pipeline import make_pipeline
from sklearn import feature_selection as fs

# ML algorithms models
from sklearn import linear_model as lm
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor 
from  xgboost import XGBRegressor

# ML Evaluations
import sklearn.metrics as sklm
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split

#Ploting
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = (pd.read_csv('df_enc.csv'))
x= df.drop(['Unnamed: 0','population','evictions'], axis=1)
y= df['evictions']


In [3]:
x.head()

,renter_occupied_households,pct_renter_occupied,median_gross_rent,median_household_income,median_property_value,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,...,UI5,UI6,UI7,UI8,UI9,UI10,UI11,UI12,YA,YB
0,6944.0,37.218,643.0,33315.0,98494.0,33.389,0.412070,0.493459,0.070193,0.002588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1224.0,31.773,517.0,43724.0,85444.0,26.477,0.838682,0.015588,0.037419,0.073493,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1725.0,21.973,671.0,37777.0,136162.0,32.500,0.873733,0.041040,0.046922,0.004503,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,18180.0,36.809,603.0,30607.0,70062.0,31.959,0.264083,0.240836,0.081067,0.377986,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,551.0,17.627,668.0,44237.0,187066.0,29.339,0.924502,0.005146,0.035753,0.014045,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [4]:
x = np.array(x)
y=np.array(y)

In [5]:
print(x.shape)
print(y.shape)

(2546, 61)
(2546,)


In [6]:
# splt train test
nr.seed(9988)
x_train, x_test, y_train, y_test = train_test_split(x,y)

## Train models selected

In [7]:
Boosting = GradientBoostingRegressor ()
Forest = RandomForestRegressor()
XGB = XGBRegressor()

In [8]:
Boosting.fit(x_train,y_train)
Forest.fit(x_train,y_train)
XGB.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [9]:
# cross validation
Results_Boosting = cross_validate(Boosting,x,y,scoring="r2",cv=5)
Results_Forest = cross_validate(Forest,x,y,scoring="r2",cv=5)
Results_XGB = cross_validate(XGB,x,y,scoring="r2",cv=5)


In [10]:
Boosting_test_scores = Results_Boosting['test_score']
Boosting_train_scores = Results_Boosting['train_score']
Forest_test_scores = Results_Forest['test_score']
Forest_train_scores = Results_Forest['train_score']
XGB_test_scores = Results_XGB['test_score']
XGB_train_scores = Results_XGB['train_score']

print('Boosting')
print(np.mean(Boosting_train_scores))
print(np.mean(Boosting_test_scores))
print('Forest')
print(np.mean(Forest_train_scores))
print(np.mean(Forest_test_scores))
print('XGB')
print(np.mean(XGB_train_scores))
print(np.mean(XGB_test_scores))

Boosting
0.9865648050625726
0.8647452971126951
Forest
0.9639694102698023
0.7990214207150143
XGB
0.9810434343922617
0.8682491103116959


In [11]:
nr.seed(123)
inside = ms.KFold(n_splits=5, shuffle = True)
nr.seed(321)
outside = ms.KFold(n_splits=5, shuffle = True)

In [12]:
nr.seed(3456)
## Define the dictionary for the grid search and the model object to search on
param_grid_Boosting = {"n_estimators": range(20,501,20)}
param_grid_Forest = {"n_estimators": range(20,501,20)}
param_grid_XGB = {"n_estimators": range(20,501,20)}

## Perform the grid search over the parameters
Grid_Boosting = ms.GridSearchCV(estimator = Boosting, param_grid = param_grid_Boosting, 
                      cv = inside, # Use the inside folds
                      scoring = 'r2',
                      return_train_score = True)

Grid_Forest = ms.GridSearchCV(estimator = Forest, param_grid = param_grid_Forest, 
                      cv = inside, # Use the inside folds
                      scoring = 'r2',
                      return_train_score = True)

Grid_XGB = ms.GridSearchCV(estimator = XGB, param_grid = param_grid_XGB, 
                      cv = inside, # Use the inside folds
                      scoring = 'r2',
                      return_train_score = True)


In [13]:
## Fit thhe cross validated grid search over the data 
Grid_Boosting.fit(x,y)
Grid_Forest.fit(x,y)
Grid_XGB.fit(x,y)



GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': range(20, 501, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='r2', verbose=0)

In [28]:
 Grid_Boosting.best_params_, Grid_Boosting.best_score_

({'n_estimators': 500}, 0.8758794120048428)

In [29]:
Grid_Forest.best_params_, Grid_Forest.best_score_

({'n_estimators': 440}, 0.8856713891453629)

In [29]:
Grid_XGB.best_params_, Grid_Forest.best_score_

({'n_estimators': 440}, 0.8856713891453629)

In [30]:
final_results_Boosting = cross_validate(Grid_Boosting.best_estimator_,x,y)
final_results_Forest = cross_validate(Grid_Forest.best_estimator_,x,y)
final_results_Forest = cross_validate(Grid_XGB.best_estimator_,x,y)

C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [31]:
test_scores_Boosting = final_results_Boosting['test_score']
train_scores_Boosting = final_results_Boosting['train_score']
print(np.mean(train_scores_Boosting))
print(np.mean(test_scores_Boosting))

0.987865579882197
0.8527832861488114


C:\Users\esteban.londono\AppData\Local\Continuum\anaconda3\envs\ML_Platzi\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [32]:
test_scores_Forest = final_results_Forest['test_score']
train_scores_Forest = final_results_Forest['train_score']
print(np.mean(train_scores_Forest))
print(np.mean(test_scores_Forest))

0.982048201682257
0.8660344372205239


In [32]:
test_scores_XGB = final_results_XGB['test_score']
train_scores_XGB = final_results_XGB['train_score']
print(np.mean(train_scores_XGB))
print(np.mean(test_scores_XGB))

0.982048201682257
0.8660344372205239


## prediction

In [ ]:
model = GradientBoostingRegressor (n_estimators=)

In [ ]:
test_enc=pd.read_csv('df_test_enc.csv')
test_enc=test_trf.drop(['Unnamed: 0'],axis=1)
test_enc= np.array(test_trf)

In [ ]:
E_Prediction=model.predict(test_enc)
E_Prediction= pd.DataFrame(AE_Prediction,columns=['evictions'])
E_Prediction.index.names=['row_id']
E_Prediction['evictions']=E_Prediction['evictions'].astype(np.int64)
E_Prediction.head()

In [ ]:
E_Prediction.to_csv('E_Prediction.csv')